# validation
대회에서 주어진 946개 그대로 사용

In [88]:
import os

val_dir_path = "D:/2022_imgdl/data/validation/images"
val_ids = os.listdir(val_dir_path)

In [89]:
# hyperparameter
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3
width_out = 128
height_out = 128

In [90]:
import numpy as np

x_val = np.zeros((len(val_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
y_val = np.zeros((len(val_ids), height_out, width_out, 1), dtype=bool)

In [91]:
import cv2

for i, id_ in enumerate(val_ids):
    img = cv2.imread('D:/2022_imgdl/data/validation/images/'+id_)
    img = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH))
    x_val[i] = img

    mask = np.zeros((height_out,width_out,1), dtype=bool)
    mask_path = "D:/2022_imgdl/data/validation/masks/"
    mask_ = cv2.imread(mask_path+id_[:6]+".png", 0)
    mask_ = cv2.resize(mask_, (height_out, width_out))
    mask_ = np.expand_dims(mask_, axis=-1)
    mask_ = np.where(mask_ >= 115, True, False)
    mask = np.maximum(mask,mask_)

    y_val[i] = mask

In [92]:
np.save("x_val.npy", x_val)
np.save("y_val.npy", y_val)

# 단순 random sampling
class imbalanced 고려하지 않고 단순히 이미지 10,000개 random sample

In [82]:
import os

train_dir_path = "D:/2022_imgdl/data/training/images"
train_ids = os.listdir(train_dir_path)

In [83]:
import random

random.seed(100)
train_ids = random.sample(train_ids, 10000)

In [84]:
# hyperparameter
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3
width_out = 128
height_out = 128

In [85]:
import numpy as np

x_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
y_train = np.zeros((len(train_ids), height_out, width_out, 1), dtype=bool)

In [86]:
import cv2

for i, id_ in enumerate(train_ids):
    img = cv2.imread('D:/2022_imgdl/data/training/images/'+id_)
    img = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH))
    x_train[i] = img

    mask = np.zeros((height_out,width_out,1), dtype=bool)
    mask_path = "D:/2022_imgdl/data/training/masks/"
    mask_ = cv2.imread(mask_path+id_[:6]+".png", 0)
    mask_ = cv2.resize(mask_, (height_out, width_out))
    mask_ = np.expand_dims(mask_, axis=-1)
    mask_ = np.where(mask_ >= 115, True, False)
    mask = np.maximum(mask,mask_)

    y_train[i] = mask

In [87]:
np.save("x_train_randomsample.npy", x_train)
np.save("y_train_randomsample.npy", y_train)

# class imbalanced 고려한 sampling
category count 했을 때 median 값 추출 -> 그 값 이상인 것 under sampling + 그 값 이하인 것 그대로 -> img 29,102장 중 10,000장 random sampling


category 별 count 결과 : (max, median, min) = (6970, 157, 55)   
157개 이상인 category -> category 당 157개로 under sampling해서 사용 (a)    
157개 미만인 category -> 모두 사용 (b)  
(a) + (b) -> 40,589개 -> 여기에서 image_id만 추출하면 29,102장  
29,102장에서 10,000장 random sampling

In [ ]:
import os

train_dir_path = "D:/2022_imgdl/data/training/images"
train_ids = os.listdir(train_dir_path)

In [48]:
# Reading annotations.json

TRAIN_ANNOTATIONS_PATH = "D:/2022_imgdl/data/training/annotations.json"
TRAIN_IMAGE_DIRECTIORY = "D:/2022_imgdl/data/training/images/"

VAL_ANNOTATIONS_PATH = "D:/2022_imgdl/data/validation/annotations.json"
VAL_IMAGE_DIRECTIORY = "D:/2022_imgdl/data/validation/images/"

In [49]:
# Reading the annotation files
import json

with open(TRAIN_ANNOTATIONS_PATH) as f:
  train_annotations_data = json.load(f)

with open(VAL_ANNOTATIONS_PATH) as f:
  val_annotations_data = json.load(f)

In [50]:
# For reading annotations file
import pycocotools
from pycocotools.coco import COCO

train_coco = COCO(TRAIN_ANNOTATIONS_PATH)
val_coco = COCO(VAL_ANNOTATIONS_PATH)

loading annotations into memory...
Done (t=5.59s)
creating index...
index created!
loading annotations into memory...
Done (t=0.07s)
creating index...
index created!


In [51]:
len(train_ids)

54392

In [52]:
# train_ids를 int형으로 추출

train_ids_int = []

for i in range(len(train_ids)):
    train_ids_int.append(train_ids[i][:6])

train_ids_int = list(map(int, train_ids_int))

In [53]:
# annotations에서 정보 추출

annos = []

for i in range(len(train_annotations_data['annotations'])):
    if train_annotations_data['annotations'][i]['image_id'] in train_ids_int:
        annos.append(train_annotations_data['annotations'][i])

In [54]:
import pandas as pd

anno_df = pd.DataFrame.from_dict(annos)
anno_df

,id,image_id,category_id,segmentation,area,bbox,iscrowd
0,184123,131072,101246,"[[169.0, 379.5, 130.0, 374.5, 112.0, 363.5, 94...",71393.0,"[61.5, 61.5, 318.0, 322.0]",0
1,184983,131072,100546,"[[354.82462890625004, 122.45733215332031, 235....",57608.0,"[93.82462890625001, 93.82462890625001, 274.552...",0
2,184129,131087,101129,"[[296.82462890625, 129.41734985351562, 106.584...",68720.0,"[62.50462890625, 62.50462890625, 347.632720947...",0
3,184986,131088,101243,"[[147.830625, 147.92533203125, 91.510624999999...",88177.0,"[4.4706249999999885, 4.4706249999999885, 380.2...",0
4,184987,131088,100133,"[[155.510625, 288.72533203125, 213.110625, 304...",67106.0,"[40.31062499999999, 8.310624999999987, 441.6, ...",0
...,...,...,...,...,...,...,...
100251,184977,131053,101363,"[[160.19592651367188, 191.64390075683596, 214....",17230.0,"[49.90640075683593, 71.24342651367188, 228.567...",0
100252,185198,131066,101178,"[[149.06, 365.97999999999996, 110.78, 227.94, ...",32875.0,"[110.78, 110.78, 255.19999999999996, 177.48]",0
100253,184122,131071,101246,"[[193.0, 412.5, 143.0, 411.5, 116.0, 399.5, 11...",86079.0,"[64.5, 64.5, 348.0, 350.0]",0
100254,185200,131071,101194,"[[357.86, 268.54, 312.62, 246.5, 253.46, 216.3...",13822.0,"[103.82000000000001, 86.42, 281.88000000000005...",0


In [55]:
# category 개수

cateList = anno_df['category_id'].unique()
len(cateList)

323

In [56]:
# category 별 개수

count = anno_df['category_id'].value_counts()
count_num = count.to_list()
print(count_num)

[6970, 2842, 2737, 2634, 2620, 2256, 2231, 2138, 2085, 1568, 1482, 1354, 1337, 1284, 1124, 1123, 1081, 958, 936, 889, 849, 833, 808, 764, 747, 666, 663, 633, 618, 610, 589, 584, 572, 562, 553, 524, 524, 505, 502, 490, 483, 482, 471, 467, 435, 434, 433, 430, 426, 413, 413, 409, 403, 401, 390, 382, 365, 363, 356, 356, 355, 355, 350, 347, 346, 345, 345, 343, 340, 335, 334, 334, 334, 329, 319, 318, 313, 310, 309, 308, 305, 304, 303, 299, 298, 298, 295, 294, 293, 289, 288, 286, 284, 283, 271, 267, 267, 262, 260, 259, 258, 258, 252, 245, 244, 244, 243, 239, 235, 222, 221, 220, 220, 220, 216, 213, 211, 210, 209, 208, 207, 207, 206, 206, 203, 201, 201, 199, 197, 193, 193, 193, 190, 190, 189, 186, 186, 183, 180, 180, 177, 177, 176, 176, 176, 176, 175, 174, 172, 168, 168, 168, 164, 164, 163, 163, 163, 161, 160, 157, 157, 157, 156, 152, 152, 150, 149, 147, 147, 146, 146, 146, 146, 145, 145, 144, 140, 138, 135, 134, 134, 133, 131, 130, 130, 129, 129, 127, 127, 127, 126, 125, 122, 121, 120, 120, 11

In [57]:
# category 별 개수 median 구하기

import numpy as np
np.median(count_num)

157.0

In [58]:
# median보다 많은 개수를 가지는 category_id 추출

count_df = count.to_frame()
idx = count_df[count_df['category_id']>=157].index
idx = idx.to_list()
len(idx)

162

In [59]:
df_undersam = anno_df[anno_df['category_id'].isin(idx)]
df_undersam['category_id'].value_counts()

101129    6970
101208    2842
100089    2737
101255    2634
101178    2620
          ... 
100324     161
100338     160
100916     157
101324     157
100495     157
Name: category_id, Length: 162, dtype: int64

In [60]:
# under sampling

X, y = df_undersam.loc[:, ['id','image_id','segmentation','area','bbox','iscrowd']], df_undersam['category_id']

In [61]:
from imblearn.under_sampling import RandomUnderSampler
X_resampled, y_resampled = RandomUnderSampler(random_state=0).fit_resample(X, y)

In [62]:
y_resampled_d = pd.Series(y_resampled)
y_resampled_d.value_counts()

101009    157
101181    157
100234    157
100314    157
100826    157
         ... 
100069    157
100133    157
100229    157
100245    157
100335    157
Name: category_id, Length: 162, dtype: int64

In [63]:
# under sampling 한 결과만 정리

df_resampled = pd.concat([X_resampled, y_resampled], axis=1)
df_resampled

,id,image_id,segmentation,area,bbox,iscrowd,category_id
0,84827,50037,"[[252.6, 262.8, 322.2, 290.4, 323.4, 226.79999...",31785.0,"[104.4, 113.39999999999999, 218.99999999999997...",0,50
1,250537,154822,"[[306.25, 150.42, 284.69, 136.7, 255.29, 138.6...",59724.0,"[10.29, 10.29, 295.96, 374.35999999999996]",0,50
2,287373,162735,"[[817.01625, 468.4715074539185, 511.7037500000...",182187.0,"[422.06400745391846, 448.19875, 394.9522425460...",0,50
3,143975,99541,"[[427.1675, 735.0799999999999, 493.6375, 633.4...",39126.0,"[151.51250000000002, 151.51250000000002, 583.5...",0,50
4,166446,117065,"[[702.4999999999999, 263.98, 177.2200000000000...",481922.0,"[99.50000000000003, 99.50000000000003, 602.999...",0,50
...,...,...,...,...,...,...,...
25429,67098,37357,"[[52.2, 306.90000000000003, 101.39999999999999...",31851.0,"[52.2, 45.0, 259.50000000000006, 260.4]",0,101363
25430,156106,109298,"[[341.325, 246.0, 226.525, 264.45, 226.525, 31...",106764.0,"[199.875, 199.875, 413.07500000000005, 422.300...",0,101363
25431,159658,112788,"[[245.00000000000003, 402.5, 190.0, 401.5, 160...",52632.0,"[74.5, 74.5, 328.0, 273.0]",0,101363
25432,320479,183857,"[[342.0, 106.0, 342.00370000000004, 105.9986, ...",248914.0,"[18.00098, 18.00098, 605.99612, 701.00128]",0,101363


In [64]:
# 기존 annotations에서, under sampling 진행한 category 데이터 삭제

for idxs in idx:
    index_idxs = anno_df[anno_df['category_id']==idxs].index
    anno_df.drop(index_idxs, inplace=True)

In [65]:
anno_df

,id,image_id,category_id,segmentation,area,bbox,iscrowd
13,185203,131097,101185,"[[226.29125, 117.28875000000002, 201.85375, 97...",2262.0,"[80.64375, 75.75625000000001, 208.684999999999...",0
15,185205,131098,101185,"[[229.22375000000002, 307.90124999999995, 189....",46315.0,"[58.161249999999995, 58.161249999999995, 256.5...",0
18,185207,131100,101185,"[[248.77375, 275.64374999999995, 244.863749999...",34560.0,"[56.68375000000002, 76.73375, 288.362499999999...",0
22,185211,131100,100107,"[[161.77625, 240.45374999999999, 153.956249999...",11322.0,"[73.30125000000002, 101.17124999999999, 226.77...",0
28,184161,131111,101219,"[[180.0, 434.5, 136.0, 434.5, 107.0, 430.5, 97...",85630.0,"[47.5, 47.5, 387.0, 305.0]",0
...,...,...,...,...,...,...,...
100238,185024,131013,100099,"[[332.23592651367187, 196.53140075683595, 318....",15112.0,"[88.02890075683594, 224.7109265136719, 244.207...",0
100241,184958,131016,100360,"[[214.50563476562502, 238.19533233642574, 279....",25205.0,"[69.465634765625, 69.465634765625, 262.6400000...",0
100242,184093,131017,100962,"[[365.85996093750003, 83.47998046875001, 325.0...",78901.0,"[81.07998046875, 131.85996093749998, 298.79999...",0
100243,184970,131017,100315,"[[64.6599609375, 309.0799987792969, 135.459960...",40192.0,"[64.6599609375, 38.2599609375, 294.82003784179...",0


In [66]:
# under sampling 한 data + 그대로 사용할 data

df_under = pd.concat([anno_df, df_resampled])
df_under

,id,image_id,category_id,segmentation,area,bbox,iscrowd
13,185203,131097,101185,"[[226.29125, 117.28875000000002, 201.85375, 97...",2262.0,"[80.64375, 75.75625000000001, 208.684999999999...",0
15,185205,131098,101185,"[[229.22375000000002, 307.90124999999995, 189....",46315.0,"[58.161249999999995, 58.161249999999995, 256.5...",0
18,185207,131100,101185,"[[248.77375, 275.64374999999995, 244.863749999...",34560.0,"[56.68375000000002, 76.73375, 288.362499999999...",0
22,185211,131100,100107,"[[161.77625, 240.45374999999999, 153.956249999...",11322.0,"[73.30125000000002, 101.17124999999999, 226.77...",0
28,184161,131111,101219,"[[180.0, 434.5, 136.0, 434.5, 107.0, 430.5, 97...",85630.0,"[47.5, 47.5, 387.0, 305.0]",0
...,...,...,...,...,...,...,...
25429,67098,37357,101363,"[[52.2, 306.90000000000003, 101.39999999999999...",31851.0,"[52.2, 45.0, 259.50000000000006, 260.4]",0
25430,156106,109298,101363,"[[341.325, 246.0, 226.525, 264.45, 226.525, 31...",106764.0,"[199.875, 199.875, 413.07500000000005, 422.300...",0
25431,159658,112788,101363,"[[245.00000000000003, 402.5, 190.0, 401.5, 160...",52632.0,"[74.5, 74.5, 328.0, 273.0]",0
25432,320479,183857,101363,"[[342.0, 106.0, 342.00370000000004, 105.9986, ...",248914.0,"[18.00098, 18.00098, 605.99612, 701.00128]",0


In [67]:
df_under['category_id'].value_counts()

101009    157
101201    157
100960    157
100992    157
101248    157
         ... 
101229     58
101189     58
101027     58
100174     55
100781     55
Name: category_id, Length: 323, dtype: int64

In [68]:
df_under = df_under.reset_index(drop=True)
df_under

,id,image_id,category_id,segmentation,area,bbox,iscrowd
0,185203,131097,101185,"[[226.29125, 117.28875000000002, 201.85375, 97...",2262.0,"[80.64375, 75.75625000000001, 208.684999999999...",0
1,185205,131098,101185,"[[229.22375000000002, 307.90124999999995, 189....",46315.0,"[58.161249999999995, 58.161249999999995, 256.5...",0
2,185207,131100,101185,"[[248.77375, 275.64374999999995, 244.863749999...",34560.0,"[56.68375000000002, 76.73375, 288.362499999999...",0
3,185211,131100,100107,"[[161.77625, 240.45374999999999, 153.956249999...",11322.0,"[73.30125000000002, 101.17124999999999, 226.77...",0
4,184161,131111,101219,"[[180.0, 434.5, 136.0, 434.5, 107.0, 430.5, 97...",85630.0,"[47.5, 47.5, 387.0, 305.0]",0
...,...,...,...,...,...,...,...
40584,67098,37357,101363,"[[52.2, 306.90000000000003, 101.39999999999999...",31851.0,"[52.2, 45.0, 259.50000000000006, 260.4]",0
40585,156106,109298,101363,"[[341.325, 246.0, 226.525, 264.45, 226.525, 31...",106764.0,"[199.875, 199.875, 413.07500000000005, 422.300...",0
40586,159658,112788,101363,"[[245.00000000000003, 402.5, 190.0, 401.5, 160...",52632.0,"[74.5, 74.5, 328.0, 273.0]",0
40587,320479,183857,101363,"[[342.0, 106.0, 342.00370000000004, 105.9986, ...",248914.0,"[18.00098, 18.00098, 605.99612, 701.00128]",0


In [73]:
imgidList = df_under['image_id'].unique()
imgidList = imgidList.tolist()
len(imgidList)

29102

In [78]:
# class imbalanced 해결한 data (29,102장)에서 10,000장만 random sampling

import random

random.seed(100)
train_ids_int = random.sample(imgidList, 10000)

train_ids = []
for i in range(len(train_ids_int)):
    train_ids.append(str(train_ids_int[i]).zfill(6) + '.jpg')

In [75]:
# hyperparameter
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3
width_out = 128
height_out = 128

In [76]:
import numpy as np

x_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
y_train = np.zeros((len(train_ids), height_out, width_out, 1), dtype=bool)

In [80]:
import cv2

for i, id_ in enumerate(train_ids):
    img = cv2.imread('D:/2022_imgdl/data/training/images/'+id_)
    img = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH))
    x_train[i] = img

    mask = np.zeros((height_out,width_out,1), dtype=bool)
    mask_path = "D:/2022_imgdl/data/training/masks/"
    mask_ = cv2.imread(mask_path+id_[:6]+".png", 0)
    mask_ = cv2.resize(mask_, (height_out, width_out))
    mask_ = np.expand_dims(mask_, axis=-1)
    mask_ = np.where(mask_ >= 115, True, False)
    mask = np.maximum(mask,mask_)

    y_train[i] = mask

In [81]:
np.save("x_train_clsimb.npy", x_train)
np.save("y_train_clsimb.npy", y_train)

(단순참고) category 정보

In [15]:
cate_df = pd.DataFrame.from_dict(train_annotations_data['categories'])
cate_df

,id,name,name_readable,supercategory
0,50,beetroot-steamed-without-addition-of-salt,"Beetroot, steamed, without addition of salt",food
1,101246,bread_wholemeal,bread_wholemeal,food
2,100546,jam,jam,food
3,101129,water,water,food
4,101243,bread,bread,food
...,...,...,...,...
318,101274,sauce_curry,sauce_curry,food
319,100031,watermelon_fresh,watermelon_fresh,food
320,100113,white_asparagus,white_asparagus,food
321,100151,cherries,cherries,food
